In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
input_path='/content/drive/My Drive/Dementia_Audio/Audio_Dementia/Control/*.wav'
output_path='/content/drive/My Drive/Dementia_Audio/Audio_Dementia/Control_AUG'

In [25]:
import numpy as np
import os
import random
import itertools
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [23]:
os.path.join(output_path, os.path.basename(f[0]))

'/content/drive/My Drive/Dementia_Audio/Audio_Dementia/AD_AUG/007-1.wav'

In [34]:
for file_in in glob.glob(input_path):
  data = librosa.core.load(file_in)[0]
  wn = np.random.randn(len(data))
  data_wn = data + 0.005*wn
  file_out = os.path.join(output_path, os.path.basename(file_in))
  librosa.output.write_wav(file_out, data_wn,librosa.core.load(file_in)[1])

In [35]:
!pip install numpy scipy
!pip install resampy tensorflow six

In [36]:
!git clone https://github.com/tensorflow/models.git


Cloning into 'models'...
remote: Enumerating objects: 41935, done.
remote: Total 41935 (delta 0), reused 0 (delta 0), pack-reused 41935
Receiving objects: 100% (41935/41935), 549.25 MiB | 29.38 MiB/s, done.
Resolving deltas: 100% (28503/28503), done.


In [37]:
# Grab the VGGish model
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0  78.1M      0  0:00:03  0:00:03 --:--:-- 78.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   636k      0 --:--:-- --:--:-- --:--:--  636k


In [38]:
!cp models/research/audioset/vggish/* .

In [39]:
!pip install tf_slim
!pip install soundfile

     |████████████████████████████████| 358kB 3.4MB/s 


In [44]:
import argparse

import os
import sys
import pickle
from glob import iglob
import glob
import re


import librosa
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm

import vggish_input
import vggish_slim
import vggish_postprocess
import vggish_params


MODEL_PARAMS = 'vggish_model.ckpt'
PCA_PARAMS = 'vggish_pca_params.npz'

input_list='/content/drive/My Drive/Dementia_Audio/Audio_Dementia/AD/*.wav'
output_path='/content/drive/My Drive/Dementia_Audio/Audio_Dementia/AD_NEW/'

def load_input(filename):

    y, sr = librosa.load(filename, sr=vggish_params.SAMPLE_RATE, mono=True)
    y = librosa.util.normalize(y)

    return vggish_input.waveform_to_examples(y, sr)


def run_model(files_in, outpath):

    pproc = vggish_postprocess.Postprocessor(PCA_PARAMS)

    with tf.Graph().as_default(), tf.compat.v1.Session() as sess:

        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, MODEL_PARAMS)
        features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)

        for file_in in glob.glob(files_in):

            file_out = os.path.join(outpath, os.path.extsep.join([re.sub('.wav', '', os.path.basename(file_in)), 'npz']))
            input_data = load_input(file_in)

            [embedding] = sess.run([embedding_tensor], feed_dict={features_tensor: input_data})

            emb_pca = pproc.postprocess(embedding)
            #return embedding, emb_pca
            np.savez(file_out, features=embedding, features_z=emb_pca)

In [45]:
input_list='/content/drive/My Drive/Dementia_Audio/Audio_Dementia/AD_AUG/*.wav'
output_path='/content/drive/My Drive/Dementia_Audio/Audio_Dementia/AD_AUG/'

run_model(input_list, output_path)

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Flatten instead.
INFO:tensorflow:Restoring parameters from vggish_model.ckpt
